# Manipulación de imágenes con CUDA

[`cuda-examples.py`](https://github.com/dusty-nv/jetson-utils/tree/master/python/examples/cuda-examples.py)

--------------

## Gestión de imágenes

### Formatos de imágenes

Aunque las API de transmisión de video y los objetos DNN (como imageNet, detectNet y segNet) esperan imágenes en formato RGB/RGBA, se definen una variedad de otros formatos para la adquisición de sensores y E/S de bajo nivel:

|Format string|[`imageFormat` enum](https://rawgit.com/dusty-nv/jetson-inference/dev/docs/html/group__imageFormat.html#ga931c48e08f361637d093355d64583406)|Data Type|Bit Depth|
|---|---|---|---|
|**RGB/RGBA**|rgb8|IMAGE_RGB8|uchar3|24|
| |rgba8|IMAGE_RGBA8|uchar4|32|
| |rgb32f|IMAGE_RGB32F|float3|96|
| |rgba32f|IMAGE_RGBA32F|float4|128|
|**BGR/BGRA**|bgr8|IMAGE_BGR8|uchar3|24|
| |bgra8|IMAGE_BGRA8|uchar4|32|
| |bgr32f|IMAGE_BGR32F|float3|96|
| |bgra32f|IMAGE_BGRA32F|float4|128|
|**YUV (4:2:2)**|yuyv|IMAGE_YUYV|uint8|16|
| |yuy2|IMAGE_YUY2|uint8|16|
| |yvyu|IMAGE_YVYU|uint8|16|
| |uyvy|IMAGE_UYVY|uint8|16|
|**YUV (4:2:0)**|i420|IMAGE_I420|uint8|12|
| |yv12	IMAGE_YV12|uint8|12|
| |nv12	IMAGE_NV12|uint8|12|
|**Bayer**|bayer-bggr|IMAGE_BAYER_BGGR|uint8|8|
| |bayer-gbrg|IMAGE_BAYER_GBRG|uint8|8|
| |bayer-grbg|IMAGE_BAYER_GRBG|uint8|8|
| |bayer-rggb|IMAGE_BAYER_RGGB|uint8|8|
|**Grayscale**|gray8|IMAGE_GRAY8|uint8|8|
| |gray32f|IMAGE_GRAY32F|float|32|

 * Bit Depth representa el número efectivo de bits por píxel
 * Para obtener especificaciones detalladas de los formatos YUV, consulte [fourcc.org](https://www.fourcc.org/yuv.php)

> **nota:** en C ++, los formatos RGB/RGBA son los únicos que deben usarse con los tipos de vector `uchar3`/`uchar4`/`float3`/`float4`. Se supone que cuando se utilizan estos tipos, las imágenes están en formato RGB/RGBA.

Para convertir imágenes entre formatos de datos y/o espacios de color, consulte la sección Conversión de color a continuación.

### Asignación de imágenes

Para asignar memoria de GPU vacía para almacenar imágenes intermedias / de salida (es decir, memoria de trabajo durante el procesamiento), use una de las funciones `cudaAllocMapped()` de C++ o Python. Tenga en cuenta que los flujos `videoSource` de entrada asignan automáticamente su propia memoria de GPU y le devuelven la última imagen, por lo que no necesita asignar su propia memoria para esos.

La memoria asignada por `cudaAllocMapped()` reside en un espacio de memoria compartida de CPU/GPU, por lo que es accesible tanto desde la CPU como desde la GPU sin necesidad de realizar una copia de memoria entre ellas (por lo tanto, también se conoce como memoria ZeroCopy).

Sin embargo, se requiere sincronización, por lo que si desea acceder a una imagen desde la CPU después de que se haya producido el procesamiento de la GPU, llame `cudaDeviceSynchronize()` primero. Para liberar la memoria en C++, use la función `cudaFreeHost()`. En Python, el recolector de basura liberará automáticamente la memoria, pero puede liberarla explícitamente con el operador `del`.

A continuación se muestra el pseudocódigo de Python y C++ para asignar/sincronizar/liberar la memoria ZeroCopy:

**Python**

```python
import jetson.utils

# allocate a 1920x1080 image in rgb8 format
img = jetson.utils.cudaAllocMapped(width=1920, height=1080, format='rgb8')

# do some processing on the GPU here
...

# wait for the GPU to finish processing
jetson.utils.cudaDeviceSynchronize()

# Python will automatically free the memory, but you can explicitly do it with 'del'
del img
```

**C++**

```cpp
#include <jetson-utils/cudaMappedMemory.h>

void* img = NULL;

// allocate a 1920x1080 image in rgb8 format
if( !cudaAllocMapped(&img, 1920, 1080, IMAGE_RGB8) )
	return false;	// memory error

// do some processing on the GPU here 
...

// wait for the GPU to finish processing
CUDA(cudaDeviceSynchronize());

// release the memory
CUDA(cudaFreeHost(img));
```

En C++, a menudo puede omitir la enumeración `imageFormat` explícita si sus punteros se escriben como `uchar3`/`uchar4`/`float3`/`float4`. A continuación se muestra funcionalmente equivalente a la asignación anterior:

```cpp
uchar3* img = NULL;	// can be uchar3 (rgb8), uchar4 (rgba8), float3 (rgb32f), float4 (rgba32f)

if( !cudaAllocMapped(&img, 1920, 1080) )
	return false;	
```

 > **nota**: al usar estos tipos de vectores, se asumirá que estas imágenes están en su respectivo espacio de color RGB/RGBA. Entonces, si usa `uchar3`/`uchar4`/`float3`/`float4` para representar una imagen que contiene datos BGR/BGRA, algunas funciones de procesamiento podrían interpretarla como RGB/RGBA a menos que especifique explícitamente el formato de imagen adecuado.

### Copiar imágenes

`cudaMemcpy()` se puede utilizar para copiar la memoria entre imágenes del mismo formato y dimensiones. `cudaMemcpy()` es una función CUDA estándar en C++, y hay una versión similar para Python en la biblioteca 'jetson.utils':

**Python**

```python
import jetson.utils

# load an image and allocate memory to copy it to
img_a = jetson.utils.loadImage("my_image.jpg")
img_b = jetson.utils.cudaAllocMapped(width=img_a.width, height=img_a.height, format=img_a.format)

# copy the image (dst, src)
jetson.utils.cudaMemcpy(img_b, img_a)

# or you can use this shortcut, which will make a duplicate
img_c = jetson.utils.cudaMemcpy(img_a)
```

**C++**

```cpp
#include <jetson-utils/cudaMappedMemory.h>
#include <jetson-utils/imageIO.h>

uchar3* img_a = NULL;
uchar3* img_b = NULL;

int width = 0;
int height = 0;

// load example image
if( !loadImage("my_image.jpg", &img_a, &width, &height) )
	return false;	// loading error
	
// allocate memory to copy it to
if( !cudaAllocMapped(&img_b, width, height) )
	return false;  // memory error
	
// copy the image (dst, src)
if( CUDA_FAILED(cudaMemcpy(img_b, img_a, width * height * sizeof(uchar3), cudaMemcpyDeviceToDevice)) )
	return false;  // memcpy error
```

### Cápsulas de Python

Cuando asigna una imagen en Python, o captura una imagen de un video con `videoSource.Capture()`, devolverá un objeto de cápsula de memoria autónomo (de tipo `<jetson.utils.cudaImage>`) que se puede pasar sin tener que copiar la memoria subyacente. El objeto `cudaImage` tiene los siguientes miembros:

```
<jetson.utils.cudaImage>
  .ptr      # memory address (not typically used)
  .size     # size in bytes
  .shape    # (height,width,channels) tuple
  .width    # width in pixels
  .height   # height in pixels
  .channels # number of color channels
  .format   # format string
  .mapped   # true if ZeroCopy
```

Para que pueda hacer cosas como `img.width` y `img.height` para acceder a las propiedades de la imagen.

#### Accediendo a datos de imagen en Python

Las imágenes CUDA también son subcriptables, lo que significa que puede indexarlas para acceder directamente a los datos de píxeles desde la CPU:

```python
for y in range(img.height):
	for x in range(img.width):
		pixel = img[y,x]    # returns a tuple, i.e. (r,g,b) for RGB formats or (r,g,b,a) for RGBA formats
		img[y,x] = pixel    # set a pixel from a tuple (tuple length must match the number of channels)
```

 > **nota**: el operador de índice de Python solo está disponible si la imagen se asignó en la memoria ZeroCopy asignada (es decir, por `cudaAllocMapped()`). De lo contrario, no se puede acceder a los datos desde la CPU y se lanzará una excepción.

La tupla de indexación utilizada para acceder a una imagen puede adoptar las siguientes formas:

 * img[y,x]- tenga en cuenta el orden de la tupla es `(y,x)`, igual que numpy
 * img[y,x,channel] - solo acceda a un canal en particular (es decir, 0 para rojo, 1 para verde, 2 para azul, 3 para alfa)
 * img[y*img.width+x] - índice 1D plano, accede a todos los canales en ese píxel

Aunque se admite el subíndice de imágenes, no se recomienda acceder individualmente a cada píxel de una imagen grande desde Python, ya que ralentizará significativamente la aplicación. Suponiendo que no esté disponible una implementación de GPU, una mejor alternativa es usar Numpy.

#### Conversión a matrices de Numpy

Puede acceder a una cápsula `cudaImage` de memoria de Numpy llamándola primero `jetson.utils.cudaToNumpy()`. La memoria subyacente no se copia y Numpy accederá a ella directamente, así que tenga en cuenta que si cambia los datos en el lugar a través de Numpy, también se cambiarán en la cápsula `cudaImage`.

Para ver un ejemplo de uso `cudaToNumpy()`, vea el ejemplo [cuda-to-numpy.py](https://github.com/dusty-nv/jetson-utils/blob/master/python/examples/cuda-to-numpy.py) de jetson-utils.

Tenga en cuenta que OpenCV espera imágenes en el espacio de color BGR, por lo que si planea usar la imagen con OpenCV, debe llamar `cv2.cvtColor()` con `cv2.COLOR_RGB2BGR` antes de usarla en OpenCV.

#### Conversión desde matrices de Numpy

Digamos que tiene una imagen en un ndarray de Numpy, quizás proporcionado por OpenCV. Como una matriz Numpy, solo será accesible desde la CPU. Puede usarlo `jetson.utils.cudaFromNumpy()` para copiarlo en la GPU (en la memoria compartida CPU/GPU ZeroCopy).

Para ver un ejemplo de uso `cudaFromNumpy()`, vea el ejemplo [cuda-from-numpy.py](https://github.com/dusty-nv/jetson-utils/blob/master/python/examples/cuda-from-numpy.py) de jetson-utils.

Tenga en cuenta que las imágenes de OpenCV están en el espacio de color BGR, por lo que si la imagen proviene de OpenCV, primero debe llamar `cv2.cvtColor()` con `cv2.COLOR_BGR2RGB`.

--------------

## Rutinas CUDA

### Conversión de color (Color Conversion)

La función `cudaConvertColor()` usa la GPU para convertir entre formatos de imagen y espacios de color. Por ejemplo, puede convertir de RGB a BGR (o viceversa), de YUV a RGB, de RGB a escala de grises, etc. También puede cambiar el tipo de datos y el número de canales (por ejemplo, RGB8 a RGBA32F). Para obtener más información sobre los diferentes formatos disponibles para convertir, consulte la sección Formatos de imagen anterior.

`cudaConvertColor()` tiene las siguientes limitaciones y conversiones no admitidas:

 * Los formatos YUV no admiten BGR/BGRA o escala de grises (solo RGB/RGBA)
 * YUV NV12, YUYV, YVYU y UYVY solo se pueden convertir a RGB/RGBA (no desde)
 * Los formatos Bayer solo se pueden convertir a RGB8 (uchar3) y RGBA8 (uchar4)

El siguiente psuedocódigo de Python/C ++ carga una imagen en RGB8 y la convierte a RGBA32F (tenga en cuenta que esto es puramente ilustrativo, ya que la imagen se puede cargar directamente como RGBA32F). Para obtener un ejemplo más completo, consulte [cuda-examples.py](https://github.com/dusty-nv/jetson-utils/blob/master/python/examples/cuda-examples.py).

**Python**

```python
import jetson.utils

# load the input image (default format is rgb8)
imgInput = jetson.utils.loadImage('my_image.jpg', format='rgb8') # default format is 'rgb8', but can also be 'rgba8', 'rgb32f', 'rgba32f'

# allocate the output as rgba32f, with the same width/height as the input
imgOutput = jetson.utils.cudaAllocMapped(width=imgInput.width, height=imgInput.height, format='rgba32f')

# convert from rgb8 to rgba32f (the formats used for the conversion are taken from the image capsules)
jetson.utils.cudaConvertColor(imgInput, imgOutput)
```

**C++**

```cpp
#include <jetson-utils/cudaColorspace.h>
#include <jetson-utils/cudaMappedMemory.h>
#include <jetson-utils/imageIO.h>

uchar3* imgInput = NULL;   // input is rgb8 (uchar3)
float4* imgOutput = NULL;  // output is rgba32f (float4)

int width = 0;
int height = 0;

// load the image as rgb8 (uchar3)
if( !loadImage("my_image.jpg", &imgInput, &width, &height) )
	return false;

// allocate the output as rgba32f (float4), with the same width/height
if( !cudaAllocMapped(&imgOutput, width, height) )
	return false;

// convert from rgb8 to rgba32f
if( CUDA_FAILED(cudaConvertColor(imgInput, IMAGE_RGB8, imgOutput, IMAGE_RGBA32F, width, height)) )
	return false;	// an error or unsupported conversion occurred
```

### Cambiar el tamaño (Resizing)

La función `cudaResize()` usa la GPU para cambiar la escala de las imágenes a un tamaño diferente (ya sea submuestreado [downsampled] o sobremuestreado [upsampled]). El siguiente psuedocode de Python/C++ carga una imagen y la redimensiona por un factor determinado (reducido a la mitad en el ejemplo). Para obtener un ejemplo más completo, consulte [cuda-examples.py](https://github.com/dusty-nv/jetson-utils/blob/master/python/examples/cuda-examples.py).

**Python**

```python
import jetson.utils

# load the input image
imgInput = jetson.utils.loadImage('my_image.jpg')

# allocate the output, with half the size of the input
imgOutput = jetson.utils.cudaAllocMapped(width=imgInput.width * 0.5, 
                                         height=imgInput.height * 0.5, 
                                         format=imgInput.format)

# rescale the image (the dimensions are taken from the image capsules)
jetson.utils.cudaResize(imgInput, imgOutput)
```

**C++**

```cpp
#include <jetson-utils/cudaResize.h>
#include <jetson-utils/cudaMappedMemory.h>
#include <jetson-utils/imageIO.h>

// load the input image
uchar3* imgInput = NULL;

int inputWidth = 0;
int inputHeight = 0;

if( !loadImage("my_image.jpg", &imgInput, &inputWidth, &inputHeight) )
	return false;

// allocate the output image, with half the size of the input
uchar3* imgOutput = NULL;

int outputWidth = inputWidth * 0.5f;
int outputHeight = inputHeight * 0.5f;

if( !cudaAllocMapped(&imgOutput, outputWidth, outputHeight) )
	return false;

// rescale the image
if( CUDA_FAILED(cudaResize(imgInput, inputWidth, inputHeight, imgOutput, outputWidth, outputHeight)) )
	return false;
```

### Recortar (Cropping)

La función `cudaCrop()` usa la GPU para recortar imágenes a una región de interés particular (ROI). El siguiente psuedocódigo de Python/C ++ carga una imagen y la recorta alrededor de la mitad central de la imagen. Para obtener un ejemplo más completo, consulte [cuda-examples.py](https://github.com/dusty-nv/jetson-utils/blob/master/python/examples/cuda-examples.py).

Tenga en cuenta que los rectángulos de ROI se proporcionan como coordenadas (left, top, right, bottom).

**Python**

```python
import jetson.utils

# load the input image
imgInput = jetson.utils.loadImage('my_image.jpg')

# determine the amount of border pixels (cropping around the center by half)
crop_factor = 0.5
crop_border = ((1.0 - crop_factor) * 0.5 * imgInput.width,
               (1.0 - crop_factor) * 0.5 * imgInput.height)

# compute the ROI as (left, top, right, bottom)
crop_roi = (crop_border[0], crop_border[1], imgInput.width - crop_border[0], imgInput.height - crop_border[1])

# allocate the output image, with the cropped size
imgOutput = jetson.utils.cudaAllocMapped(width=imgInput.width * crop_factor,
                                         height=imgInput.height * crop_factor,
                                         format=imgInput.format)

# crop the image to the ROI
jetson.utils.cudaCrop(imgInput, imgOutput, crop_roi)
```

**C++**

```cpp
#include <jetson-utils/cudaCrop.h>
#include <jetson-utils/cudaMappedMemory.h>
#include <jetson-utils/imageIO.h>

// load the input image
uchar3* imgInput = NULL;

int inputWidth = 0;
int inputHeight = 0;

if( !loadImage("my_image.jpg", &imgInput, &inputWidth, &inputHeight) )
	return false;

// determine the amount of border pixels (cropping around the center by half)
const float crop_factor = 0.5
const int2  crop_border = make_int2((1.0f - crop_factor) * 0.5f * inputWidth,
                                    (1.0f - crop_factor) * 0.5f * inputHeight);

// compute the ROI as (left, top, right, bottom)
const int4 crop_roi = make_int4(crop_border.x, crop_border.y, inputWidth - crop_border.x, inputHeight - crop_border.y);

// allocate the output image, with half the size of the input
uchar3* imgOutput = NULL;

if( !cudaAllocMapped(&imgOutput, inputWidth * crop_factor, inputHeight * cropFactor) )
	return false;

// crop the image
if( CUDA_FAILED(cudaCrop(imgInput, imgOutput, crop_roi, inputWidth, inputHeight)) )
	return false;
```

### Normalización (Normalization)

La función `cudaNormalize()` usa la GPU para cambiar el rango de intensidades de píxeles en una imagen. Por ejemplo, convierta una imagen con valores de píxeles entre [0,1] para tener valores de píxeles entre [0,255]. Otro rango común de valores de píxeles se encuentra entre [-1,1].

 > **nota**: todas las demás funciones en jetson-inference y jetson-utils esperan imágenes con rangos de píxeles entre [0,255], por lo que normalmente no necesitaría usar `cudaNormalize()`, pero está disponible en caso de que esté trabajando con datos de una fuente o destino alternativo.

El siguiente psuedocódigo de Python/C++ carga una imagen y la normaliza de [0,255] a [0,1].

**Python**

```python
import jetson.utils

# load the input image (its pixels will be in the range of 0-255)
imgInput = jetson.utils.loadImage('my_image.jpg')

# allocate the output image, with the same dimensions as input
imgOutput = jetson.utils.cudaAllocMapped(width=imgInput.width, height=imgInput.height, format=imgInput.format)

# normalize the image from [0,255] to [0,1]
jetson.utils.cudaNormalize(imgInput, (0,255), imgOutput, (0,1))
```

**C++**

```cpp
#include <jetson-utils/cudaNormalize.h>
#include <jetson-utils/cudaMappedMemory.h>
#include <jetson-utils/imageIO.h>

uchar3* imgInput = NULL;
uchar3* imgOutput = NULL;

int width = 0;
int height = 0;

// load the input image (its pixels will be in the range of 0-255)
if( !loadImage("my_image.jpg", &imgInput, &width, &height) )
	return false;

// allocate the output image, with the same dimensions as input
if( !cudaAllocMapped(&imgOutput, width, height) )
	return false;

// normalize the image from [0,255] to [0,1]
CUDA(cudaNormalize(imgInput, make_float2(0,255),
                   imgOutput, make_float2(0,1),
                   width, height));
```

### Overlay

La función `cudaOverlay()` usa la GPU para compostar una imagen de entrada sobre una imagen de salida en una ubicación particular. Las operaciones de superposición se suelen llamar en secuencia para formar una combinación de varias imágenes juntas.

El siguiente psuedocódigo de Python/C++ carga dos imágenes y las compone juntas una al lado de la otra en una imagen de salida.

**Python**

```python
import jetson.utils

# load the input images
imgInputA = jetson.utils.loadImage('my_image_a.jpg')
imgInputB = jetson.utils.loadImage('my_image_b.jpg')

# allocate the output image, with dimensions to fit both inputs side-by-side
imgOutput = jetson.utils.cudaAllocMapped(width=imgInputA.width + imgInputB.width, 
                                         height=max(imgInputA.height, imgInputB.height),
                                         format=imgInputA.format)

# compost the two images (the last two arguments are x,y coordinates in the output image)
jetson.utils.cudaOverlay(imgInputA, imgOutput, 0, 0)
jetson.utils.cudaOverlay(imgInputB, imgOutput, imgInputA.width, 0)
```

**C++**

```cpp
#include <jetson-utils/cudaOverlay.h>
#include <jetson-utils/cudaMappedMemory.h>
#include <jetson-utils/imageIO.h>

#include <algorithm>  // for std::max()

uchar3* imgInputA = NULL;
uchar3* imgInputB = NULL;
uchar3* imgOutput = NULL;

int2 dimsA = make_int2(0,0);
int2 dimsB = make_int2(0,0);

// load the input images
if( !loadImage("my_image_a.jpg", &imgInputA, &dimsA.x, &dimsA.y) )
	return false;

if( !loadImage("my_image_b.jpg", &imgInputB, &dimsB.x, &dimsB.y) )
	return false;

// allocate the output image, with dimensions to fit both inputs side-by-side
const int2 dimsOutput = make_int2(dimsA.x + dimsB.x, std::max(dimsA.y, dimsB.y));

if( !cudaAllocMapped(&imgOutput, dimsOutput.x, dimsOutput.y) )
	return false;

// compost the two images (the last two arguments are x,y coordinates in the output image)
CUDA(cudaOverlay(imgInputA, dimsA, imgOutput, dimsOutput, 0, 0));
CUDA(cudaOverlay(imgInputB, dimsB, imgOutput, dimsOutput, dimsA.x, 0));
```

### Drawing Shapes

`cudaDraw.h` define varias funciones para dibujar formas básicas, incluidos círculos, líneas y rectángulos.

A continuación, se muestran códigos sencillos de Python y C++ para usarlos; consulte en [cuda-examples.py](https://github.com/dusty-nv/jetson-utils/blob/master/python/examples/cuda-examples.py) un ejemplo de funcionamiento.

**Python**

```python
# load the input image
input = jetson.utils.loadImage("my_image.jpg")

# cudaDrawCircle(input, (cx,cy), radius, (r,g,b,a), output=None)
jetson.utils.cudaDrawCircle(input, (50,50), 25, (0,255,127,200))

# cudaDrawRect(input, (left,top,right,bottom), (r,g,b,a), output=None)
jetson.utils.cudaDrawRect(input, (200,25,350,250), (255,127,0,200))

# cudaDrawLine(input, (x1,y1), (x2,y2), (r,g,b,a), line_width, output=None)
jetson.utils.cudaDrawLine(input, (25,150), (325,15), (255,0,200,200), 10)
```

 > **nota**: si la entrada opciona `output` no se especifica, la operación se realizará en el lugar de la imagen de entrada.

**C++**

```cpp
#include <jetson-utils/cudaDraw.h>
#include <jetson-utils/imageIO.h>

uchar3* img = NULL;
int width = 0;
int height = 0;

// load example image
if( !loadImage("my_image.jpg", &img, &width, &height) )
	return false;	// loading error
	
// see cudaDraw.h for definitions
CUDA(cudaDrawCircle(img, width, height, 50, 50, 25, make_float4(0,255,127,200)));
CUDA(cudaDrawRect(img, width, height, 200, 25, 350, 250, make_float4(255,127,0,200)));
CUDA(cudaDrawLine(img, width, height, 25, 150, 325, 15, make_float4(255,0,200,200), 10));
```